In [0]:
CREATE TABLE EMP (
    EMPNO INT PRIMARY KEY,
    ENAME VARCHAR(50) NOT NULL,
    JOB VARCHAR(30),
    MGR INT,
    HIREDATE DATE,
    SAL DECIMAL(10,2),
    COMM DECIMAL(10,2),
    DEPTNO INT
);

In [0]:
select * from emp

In [0]:
df = spark.read.table('emp')
display(df)

Q3. Update salary by 10% for all employees in department 10.****

In [0]:
update emp set 
sal = sal *1.1 
where JOB = 'manager'

In [0]:
select * from emp

In [0]:
select * from dept

**Give a 20% bonus to all employees in SALES department.**

In [0]:
update emp 
set SAL = sal*1.2 
where DEPTNO = (select deptno from dept where dname = 'sales');

**Retrieve all employees working in the 'SALES' department.**

In [0]:
select e.* from emp e
join dept d on e.deptno = d.deptno
where d.dname = 'sales'

**Find the highest paid employee in each department.**

In [0]:
select * from emp

In [0]:
select * from dept

In [0]:
select deptno, max(sal) from emp group by deptno

In [0]:
 SELECT DEPTNO, ENAME, SAL
FROM EMP e
WHERE SAL = (
    SELECT MAX(SAL) FROM EMP WHERE DEPTNO = e.DEPTNO
);

**Q5. Display employees who earn more than the average salary of their department.**

In [0]:
SELECT EMPNO, ENAME, SAL, DEPTNO
FROM (
    SELECT EMPNO, ENAME, SAL, DEPTNO,
           AVG(SAL) OVER (PARTITION BY DEPTNO) AS DEPT_AVG
    FROM EMP
) t
WHERE SAL > DEPT_AVG;

In [0]:
SELECT e.EMPNO, e.ENAME, e.SAL, e.DEPTNO
FROM EMP e
WHERE SAL > (
    SELECT AVG(SAL) FROM EMP WHERE DEPTNO = e.DEPTNO
);


**Q4. Find departments that don’t have any employees.**

In [0]:
SELECT d.DEPTNO, d.DNAME
FROM DEPT d
LEFT JOIN EMP e ON d.DEPTNO = e.DEPTNO
WHERE e.EMPNO IS NULL;

**Q5. Show employee name, job, salary, and their manager’s name.**


In [0]:
SELECT e.ENAME AS Employee, e.JOB, e.SAL, m.ENAME AS Manager
FROM EMP e
LEFT JOIN EMP m ON e.MGR = m.EMPNO;

In [0]:
SELECT e.ENAME AS Employee, 
       e.JOB, 
       e.SAL,
       (SELECT m.ENAME 
        FROM EMP m 
        WHERE m.EMPNO = e.MGR) AS Manager
FROM EMP e;

**Q6. Find the 2nd highest salary.**

In [0]:
select max(sal) from emp
where sal<(select max(SAL)from emp
);

**Q7. List employees who joined in the last 1 year.**


In [0]:
select * from emp where HIREDATE >= ADD_MONTHS(HIREDATE,-12)

**Q8. Find employees who have the same job as 'ALLEN'.**

In [0]:
select * from emp where job like(
  select job where ename = 'allen'
)

**Q9. Show department-wise total salary and number of employees, order by salary desc.**

In [0]:
select deptno, sum(sal) as Total_sal, count(*) as total_employees 
from emp
group by deptno
order by Total_sal DESC;

**Q10. Find employees who earn more than their manager.**

In [0]:
select * from emp

In [0]:
SELECT e.EMPNO, e.ENAME, m.ENAME AS MANAGER_NAME, e.SAL, m.SAL AS MANAGER_SALARY 
FROM EMP e 
JOIN EMP m ON e.MGR = m.EMPNO 
WHERE e.SAL > m.SAL;

In [0]:
SELECT e.EMPNO, e.ENAME, m.ENAME AS MANAGER_NAME, e.SAL, m.SAL AS MANAGER_SALARY 
FROM EMP e 
JOIN EMP m ON e.MGR = m.EMPNO 
WHERE e.SAL > m.SAL;

In [0]:
SELECT e.ENAME AS Employee, e.SAL, m.ENAME AS Manager, m.SAL AS Manager_Salary
FROM EMP e
JOIN EMP m ON e.MGR = m.EMPNO
WHERE e.SAL > m.SAL;

**Top 3 salaries across the company (ties included)**

In [0]:
SELECT EMPNO, ENAME, SAL
FROM (
SELECT e.*, DENSE_RANK() OVER (ORDER BY SAL DESC) AS dr
FROM EMP e
)
WHERE dr <= 3
ORDER BY SAL DESC, EMPNO;

**Highest paid employee(s) per department**

In [0]:
SELECT DEPTNO, EMPNO, ENAME, SAL
FROM (
SELECT e.*,
DENSE_RANK() OVER (PARTITION BY DEPTNO ORDER BY SAL DESC) AS dr
 FROM EMP e
)
WHERE dr = 1
ORDER BY DEPTNO, EMPNO;

**Running total of salaries ordered by hire date**

In [0]:
SELECT EMPNO, ENAME, HIREDATE, SAL,
SUM(SAL) OVER (ORDER BY HIREDATE, EMPNO) AS running_total_sal
FROM EMP
ORDER BY HIREDATE, EMPNO;

**Rank of employees by salary per department**

In [0]:
SELECT DEPTNO, EMPNO, ENAME, SAL,
RANK()        OVER (PARTITION BY DEPTNO ORDER BY SAL DESC)        AS sal_rank,
DENSE_RANK()  OVER (PARTITION BY DEPTNO ORDER BY SAL DESC)        AS sal_dense_rank,
ROW_NUMBER()  OVER (PARTITION BY DEPTNO ORDER BY SAL DESC, EMPNO) AS sal_rownum
FROM EMP
ORDER BY DEPTNO, sal_rank, EMPNO;